In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression

df = pd.read_csv("Drought.csv")

C:\Users\tariq.aziz\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\_param_validation.py:14: UserWarning: A NumPy version >=1.22.4 and <2.3.0 is required for this version of SciPy (detected version 1.21.5)
  from scipy.sparse import csr_matrix, issparse


In [2]:
df

,PRECTOT,PS,QV2M,T2M,T2MDEW,T2MWET,T2M_MAX,T2M_MIN,T2M_RANGE,TS,WS10M,WS10M_MAX,WS10M_MIN,WS10M_RANGE,WS50M,WS50M_MAX,WS50M_MIN,WS50M_RANGE,score
0,12.651429,99.061429,6.207143,8.721429,5.657143,5.668571,13.967143,3.277143,10.692857,8.680000,3.647143,5.291429,2.315714,2.975714,6.232857,8.801429,4.027143,4.775714,0.0000
1,5.494286,99.215714,14.404286,26.344286,19.338571,19.338571,32.615714,20.750000,11.865714,26.238571,1.772857,2.394286,1.121429,1.277143,3.520000,4.837143,2.190000,2.648571,0.4157
2,3.937143,99.857143,3.668571,2.637143,-1.215714,-1.162857,8.901429,-2.872857,11.774286,2.582857,1.841429,2.782857,0.987143,1.797143,3.852857,5.534286,1.754286,3.781429,0.0000
3,0.515714,100.082857,15.178571,28.222857,20.382857,20.384286,34.538571,21.947143,12.592857,28.114286,1.737143,2.474286,0.900000,1.575714,3.252857,4.964286,1.760000,3.207143,0.8344
4,7.432857,99.010000,8.411429,13.994286,10.678571,10.680000,20.364286,7.641429,12.724286,14.008571,2.761429,4.188571,1.614286,2.574286,5.604286,7.730000,3.474286,4.258571,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9993,1.028571,100.251429,11.711429,25.627143,15.672857,15.678571,32.341429,20.055714,12.285714,25.780000,2.935714,4.098571,1.735714,2.362857,5.557143,7.265714,3.415714,3.851429,1.9925
9994,4.641429,99.601429,10.634286,19.408571,14.427143,14.427143,25.182857,13.321429,11.861429,19.067143,1.820000,2.611429,0.992857,1.622857,3.985714,5.642857,2.144286,3.500000,0.0000
9995,2.207143,93.388571,3.942857,7.725714,-1.560000,-1.438571,15.664286,1.388571,14.277143,7.060000,4.271429,6.272857,1.770000,4.502857,6.254286,9.165714,3.002857,6.162857,0.0000
9996,0.022857,99.284286,10.301429,17.640000,13.335714,13.335714,23.814286,11.011429,12.802857,17.551429,2.230000,2.994286,1.535714,1.457143,4.214286,5.935714,2.431429,3.501429,0.0000


In [3]:
df.shape

(9998, 19)

In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression

df = pd.read_csv("Drought.csv")

def reclassify_drought(score):
    if score == 0:
        return 0   # Low
    elif score in [1, 2, 3]:
        return 1   # Medium
    else:
        return 2   # Extreme

df['drought_level'] = df['score'].apply(reclassify_drought)

X = df.drop(columns=['score', 'drought_level'])
y = df['drought_level']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)


xgb_model = XGBClassifier(
    n_estimators=500,
    learning_rate=0.05,
    max_depth=10,
    objective='multi:softprob',
    num_class=3,
    random_state=42,
    verbosity=0
)


rf_model = RandomForestClassifier(
    n_estimators=500,
    max_depth=15,
    random_state=42,
    n_jobs=1
)

meta_model = LogisticRegression(max_iter=1000, class_weight='balanced', random_state=42)

stacking_ensemble = StackingClassifier(
    estimators=[
        ('xgb', xgb_model),
        ('rf', rf_model)
    ],
    final_estimator=meta_model,
    cv=5,
    n_jobs=1,         
    passthrough=True
)

stacking_ensemble.fit(X_train, y_train)

y_pred = stacking_ensemble.predict(X_test)
acc = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred, target_names=["Low", "Medium", "Extreme"])

print("Stacking Ensemble Accuracy:", acc)
print("Stacking Ensemble Classification Report:")
print(report)


C:\Users\tariq.aziz\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Stacking Ensemble Accuracy: 0.5965
Stacking Ensemble Classification Report:
              precision    recall  f1-score   support

         Low       0.71      0.74      0.73       947
      Medium       0.26      0.45      0.33       265
     Extreme       0.67      0.47      0.55       788

    accuracy                           0.60      2000
   macro avg       0.55      0.55      0.54      2000
weighted avg       0.64      0.60      0.61      2000



In [16]:
import joblib
joblib.dump(stacking_ensemble, 'stacking_ensemble_model.pkl')

['stacking_ensemble_model.pkl']

In [35]:
import pandas as pd

feature_names = [
    "PRECTOT", "PS", "QV2M", "T2M", "T2MDEW", "T2MWET",
    "T2M_MAX", "T2M_MIN", "T2M_RANGE", "TS",
    "WS10M", "WS10M_MAX", "WS10M_MIN", "WS10M_RANGE",
    "WS50M", "WS50M_MAX", "WS50M_MIN", "WS50M_RANGE"
]

def predict_drought_level(input_values, feature_names=feature_names):
    """
    Given a list of 18 feature values (ordered as in feature_names),
    returns the predicted drought level as a label: "Low", "Medium", or "Extreme".
    
    This function creates a one-row DataFrame and then uses the trained 
    stacking_ensemble model (assumed to be defined) to predict the drought level.
    """
    input_df = pd.DataFrame([input_values], columns=feature_names)
    
    prediction = stacking_ensemble.predict(input_df)
    
    label_map = {0: "Low", 1: "Medium", 2: "Extreme"}
    predicted_label = label_map[int(prediction[0])]
    return predicted_label

new_values = [323, 533, 2, 13, 23, 1, 2, 1, 
              15, 1, 5, 4, 233, 32, 613, 75, 4, 33]

predicted_level = predict_drought_level(new_values)
print("Predicted Drought Level:", predicted_level)

Predicted Drought Level: Extreme
